<a href="https://colab.research.google.com/github/aekanun2020/2022-PSU-Data-Science-for-Tourism-Course2/blob/main/Colab_PySpark_Clustering%2Bof%2BNetwork%2BInteraction_Demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Setting up Spark Environment

In [1]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://archive.apache.org/dist/spark/spark-3.1.2/spark-3.1.2-bin-hadoop2.7.tgz
!tar xf spark-3.1.2-bin-hadoop2.7.tgz
!pip install -q findspark

In [2]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.1.2-bin-hadoop2.7"

In [3]:
import findspark
findspark.init()
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()

In [4]:
from pyspark.sql import SparkSession

In [5]:
sc = spark.sparkContext

In [6]:
sc

<SparkContext master=local[*] appName=pyspark-shell>

In [7]:
! rm -rf ./kddcup.data*

In [8]:
! rm -rf kddcup.names*

http://kdd.ics.uci.edu/databases/kddcup99/

In [9]:
from pyspark.sql import Row

In [10]:
max_k = 20
data_file = '/rawzone/example/kddcup.data'

In [11]:
! wget http://kdd.ics.uci.edu/databases/kddcup99/kddcup.data.gz

--2022-03-20 07:38:23--  http://kdd.ics.uci.edu/databases/kddcup99/kddcup.data.gz
Resolving kdd.ics.uci.edu (kdd.ics.uci.edu)... 128.195.1.86
Connecting to kdd.ics.uci.edu (kdd.ics.uci.edu)|128.195.1.86|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 18115902 (17M) [application/x-gzip]
Saving to: ‘kddcup.data.gz’

kddcup.data.gz      100%[===================>]  17.28M  9.34MB/s    in 1.8s    

2022-03-20 07:38:25 (9.34 MB/s) - ‘kddcup.data.gz’ saved [18115902/18115902]



In [12]:
! gunzip kddcup.data.gz
! ls ./kddcup.data* -lh

-rw-r--r-- 1 root root 709M Jun 26  2007 ./kddcup.data


In [13]:
! head -3 ./kddcup.data

0,tcp,http,SF,215,45076,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,1,0.00,0.00,0.00,0.00,1.00,0.00,0.00,0,0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,normal.
0,tcp,http,SF,162,4528,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,2,2,0.00,0.00,0.00,0.00,1.00,0.00,0.00,1,1,1.00,0.00,1.00,0.00,0.00,0.00,0.00,0.00,normal.
0,tcp,http,SF,236,1228,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,1,0.00,0.00,0.00,0.00,1.00,0.00,0.00,2,2,1.00,0.00,0.50,0.00,0.00,0.00,0.00,0.00,normal.


In [14]:
! wc -l kddcup.data

4898431 kddcup.data


In [15]:
! wget http://kdd.ics.uci.edu/databases/kddcup99/kddcup.names

--2022-03-20 07:38:30--  http://kdd.ics.uci.edu/databases/kddcup99/kddcup.names
Resolving kdd.ics.uci.edu (kdd.ics.uci.edu)... 128.195.1.86
Connecting to kdd.ics.uci.edu (kdd.ics.uci.edu)|128.195.1.86|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1307 (1.3K)
Saving to: ‘kddcup.names’

kddcup.names        100%[===================>]   1.28K  --.-KB/s    in 0s      

2022-03-20 07:38:30 (109 MB/s) - ‘kddcup.names’ saved [1307/1307]



In [16]:
! head -30 kddcup.names

back,buffer_overflow,ftp_write,guess_passwd,imap,ipsweep,land,loadmodule,multihop,neptune,nmap,normal,perl,phf,pod,portsweep,rootkit,satan,smurf,spy,teardrop,warezclient,warezmaster.
duration: continuous.
protocol_type: symbolic.
service: symbolic.
flag: symbolic.
src_bytes: continuous.
dst_bytes: continuous.
land: symbolic.
wrong_fragment: continuous.
urgent: continuous.
hot: continuous.
num_failed_logins: continuous.
logged_in: symbolic.
num_compromised: continuous.
root_shell: continuous.
su_attempted: continuous.
num_root: continuous.
num_file_creations: continuous.
num_shells: continuous.
num_access_files: continuous.
num_outbound_cmds: continuous.
is_host_login: symbolic.
is_guest_login: symbolic.
count: continuous.
srv_count: continuous.
serror_rate: continuous.
srv_serror_rate: continuous.
rerror_rate: continuous.
srv_rerror_rate: continuous.
same_srv_rate: continuous.


In [17]:
fieldname_raw_rdd = sc.textFile('kddcup.names')

In [18]:
all_attack = str(fieldname_raw_rdd.take(1)[0])

In [19]:
fieldname_rdd = fieldname_raw_rdd.\
filter(lambda x: x != all_attack).\
map(lambda y: y.split(':')).\
map(lambda z:(z[0],z[1]))

In [20]:
from pyspark.sql.types import *

In [21]:
def createschema(item):
    if item[1] == ' continuous.': 
        return StructField(item[0], FloatType(), True)
    if item[1] == ' symbolic.': 
        return StructField(item[0], StringType(), True)
    

In [22]:
fieldname_list = fieldname_rdd.map(createschema).collect()

In [23]:
fieldname_list.append(StructField('attack', StringType(), True))

In [24]:
schema = StructType(fieldname_list)

In [25]:
from pyspark.ml.feature import OneHotEncoder, StringIndexer
from pyspark.ml.feature import VectorAssembler
from pyspark.mllib.clustering import KMeans, KMeansModel
from pyspark.ml.feature import StringIndexer, VectorAssembler,\
OneHotEncoder
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator
from pyspark.ml.evaluation import MulticlassClassificationEvaluator,\
BinaryClassificationEvaluator
from pyspark.ml.classification import LogisticRegression, GBTClassifier,\
NaiveBayes, RandomForestClassifier
from pyspark.ml import Pipeline
from pyspark.ml.clustering import *

In [26]:
from pyspark import SQLContext

In [27]:
sqlContext = SQLContext(sc)

In [28]:
raw_df = sqlContext.read.format('com.databricks.spark.csv').\
options(header='false', inferschema='true').\
schema(schema).load('kddcup.data')

In [29]:
raw_df1 = raw_df.repartition(60)

In [30]:
raw_df2 = raw_df1.cache()

In [31]:
raw_df2.printSchema()

root
 |-- duration: float (nullable = true)
 |-- protocol_type: string (nullable = true)
 |-- service: string (nullable = true)
 |-- flag: string (nullable = true)
 |-- src_bytes: float (nullable = true)
 |-- dst_bytes: float (nullable = true)
 |-- land: string (nullable = true)
 |-- wrong_fragment: float (nullable = true)
 |-- urgent: float (nullable = true)
 |-- hot: float (nullable = true)
 |-- num_failed_logins: float (nullable = true)
 |-- logged_in: string (nullable = true)
 |-- num_compromised: float (nullable = true)
 |-- root_shell: float (nullable = true)
 |-- su_attempted: float (nullable = true)
 |-- num_root: float (nullable = true)
 |-- num_file_creations: float (nullable = true)
 |-- num_shells: float (nullable = true)
 |-- num_access_files: float (nullable = true)
 |-- num_outbound_cmds: float (nullable = true)
 |-- is_host_login: string (nullable = true)
 |-- is_guest_login: string (nullable = true)
 |-- count: float (nullable = true)
 |-- srv_count: float (nullable = 

# Data Understanding with EDA

In [32]:
raw_df2.describe().toPandas().transpose()

,0,1,2,3,4
summary,count,mean,stddev,min,max
duration,4898431,48.34243046395876,723.3298112546697,0.0,58329.0
protocol_type,4898431,None,None,icmp,udp
service,4898431,None,None,IRC,whois
flag,4898431,None,None,OTH,SH
src_bytes,4898431,1834.6211678800823,941431.070365538,0.0,1.3799639E9
dst_bytes,4898431,1093.6228200417645,645012.3396853015,0.0,1.30993741E9
land,4898431,5.716116037972159E-6,0.00239083343855467,0,1
wrong_fragment,4898431,6.487791703098401E-4,0.04285433675493735,0.0,3.0
urgent,4898431,7.961733052889793E-6,0.007215083685169107,0.0,14.0


In [33]:
raw_df2.count()

4898431

In [34]:
nolabel_df = raw_df2.drop('attack').drop('service')

In [35]:
nolabel_df.count()

4898431

In [36]:
nolabel_df.dropna().count()

4898431

# Modeling with ML Pipeline and Unsupervised Learning

In [37]:
protocol_typeIndexer = StringIndexer(inputCol="protocol_type", \
                                     outputCol="protocol_typeIndex")
protocol_typeVector = OneHotEncoder(inputCol='protocol_typeIndex', \
                                    outputCol='protocol_typeVec')

flagIndexer = StringIndexer(inputCol="flag", outputCol="flagIndex")
flagVector = OneHotEncoder(inputCol='flagIndex', outputCol='flagVec')

In [38]:
assembler = VectorAssembler\
(inputCols=['duration', 'dst_bytes','wrong_fragment', 'urgent', \
            'hot', 'num_failed_logins', 'num_compromised', 'root_shell', \
            'su_attempted', 'num_root', 'num_file_creations', 'num_shells', \
            'num_access_files', 'num_outbound_cmds', 'count', 'srv_count', \
            'serror_rate', 'srv_serror_rate', 'rerror_rate', \
            'srv_rerror_rate', 'same_srv_rate', 'diff_srv_rate', \
            'srv_diff_host_rate', 'dst_host_count', 'dst_host_srv_count', \
            'dst_host_same_srv_rate', 'dst_host_diff_srv_rate', \
            'dst_host_same_src_port_rate', 'dst_host_srv_diff_host_rate', \
            'dst_host_serror_rate', 'dst_host_srv_serror_rate', \
            'dst_host_rerror_rate', 'dst_host_srv_rerror_rate',\
            'protocol_typeVec','flagVec'],outputCol='features')

In [39]:
kmeans = KMeans(featuresCol='features',\
                k=20,predictionCol='prediction',\
                maxIter=20,initMode="random", \
                initSteps=5)

In [40]:
pipeline = Pipeline().\
setStages([protocol_typeIndexer,protocol_typeVector,\
           flagIndexer,flagVector,assembler,kmeans])

In [41]:
kMeansPredictionModel = pipeline.fit(nolabel_df)

In [42]:
predictionResult = kMeansPredictionModel.transform(raw_df2)

# Model Evaluation - ประเมินจากมุมมองทางธุรกิจ

In [43]:
from pyspark.sql import functions as sparkf

##### Data Scientist ดูว่า Group ID ไหน ที่มี Meaning เป็นประโยชน์กับธุรกิจ

In [44]:
predictionResult.select(['attack','prediction'])\
.groupBy('attack','prediction').count().orderBy(sparkf.col('prediction'),ascending=False).show(100)

+----------------+----------+-------+
|          attack|prediction|  count|
+----------------+----------+-------+
|         normal.|        13|   7804|
|    warezclient.|        13|     22|
|      portsweep.|        13|    258|
|    warezmaster.|        12|      1|
|           back.|        12|     19|
|buffer_overflow.|        12|     25|
|          satan.|        12|      1|
|   guess_passwd.|        12|      1|
|     loadmodule.|        12|      3|
|        rootkit.|        12|      2|
|         normal.|        12| 189004|
|      ftp_write.|        12|      1|
|    warezclient.|        12|    275|
|           perl.|        12|      3|
|    warezclient.|        11|     70|
|      portsweep.|        11|     97|
|         normal.|        11|  30764|
|        neptune.|        10| 602004|
|          satan.|        10|  13635|
|         normal.|        10|    208|
|      portsweep.|        10|    533|
|           nmap.|        10|      1|
|       teardrop.|        10|     31|
|          s

##### เลือกกลุ่มและทำความเข้าใจความหมายของกลุ่ม

In [45]:
potentialAttack_group = 1

##### กลุ่มหมายเลข ... นี้ (1) บรรจุ data point เกินครึ่งของ data ทั้งหมด และ (2) น่าสังเกตว่า data point ในกลุ่มนี้เกิน 95% เป็น smurf อีกทั้ง (3) smurf เกือบทั้งหมดอยู่ในกลุ่มนี้ 

In [46]:
predictionResult.select(['attack','prediction'])\
.groupBy('attack','prediction').count()\
.filter(sparkf.col('prediction') == potentialAttack_group)\
.orderBy(sparkf.col('prediction'),ascending=False).show(100)

+-------+----------+-------+
| attack|prediction|  count|
+-------+----------+-------+
|normal.|         1|     19|
| smurf.|         1|2628267|
+-------+----------+-------+



##### พิสูจน์

In [47]:
predictionResult.count()

4898431

In [48]:
predictionResult.select(['attack','prediction'])\
.filter(sparkf.col('prediction') == potentialAttack_group)\
.agg(sparkf.count('prediction').alias('ข้อมูลในกลุ่มนี้ทั้งหมด')\
     ,((100/predictionResult.count())*sparkf.count('prediction')).alias('(1) ข้อมูลในกลุ่มนี้คิดเป็นกี่ % ของข้อมูลทั้งหมด'))\
.show()

+-----------------------+-------------------------------------------------+
|ข้อมูลในกลุ่มนี้ทั้งหมด|(1) ข้อมูลในกลุ่มนี้คิดเป็นกี่ % ของข้อมูลทั้งหมด|
+-----------------------+-------------------------------------------------+
|                2628286|                                53.65567056063462|
+-----------------------+-------------------------------------------------+



In [49]:
predictionResult.select(['attack','prediction'])\
.groupBy('attack','prediction')\
.agg(sparkf.round((100/predictionResult.filter(sparkf.col('prediction') == potentialAttack_group).count())\
                  *sparkf.count('prediction'),5).alias('(2) เกือบทั้งหมดในกลุ่มนี้เป็น smurf'))\
.filter(sparkf.col('prediction') == potentialAttack_group)\
.show()

+-------+----------+------------------------------------+
| attack|prediction|(2) เกือบทั้งหมดในกลุ่มนี้เป็น smurf|
+-------+----------+------------------------------------+
|normal.|         1|                              7.2E-4|
| smurf.|         1|                            99.99928|
+-------+----------+------------------------------------+



In [50]:
predictionResult.select(['attack','prediction'])\
.filter(sparkf.col('prediction') == potentialAttack_group)\
.filter(sparkf.col('attack').startswith('smurf'))\
.agg(sparkf.count('prediction').alias('จำนวน smurf ในกลุ่มนี้')\
     ,((100/predictionResult.filter(sparkf.col('attack').startswith('smurf'))\
        .count())*sparkf.count('prediction')).alias('(3) smurf ในกลุ่มนี้คิดเป็นกี่ % ของ smurf ทั้งหมด'))\
.show()

+----------------------+--------------------------------------------------+
|จำนวน smurf ในกลุ่มนี้|(3) smurf ในกลุ่มนี้คิดเป็นกี่ % ของ smurf ทั้งหมด|
+----------------------+--------------------------------------------------+
|               2628267|                                 93.60305226066869|
+----------------------+--------------------------------------------------+



In [51]:
! wget http://kdd.ics.uci.edu/databases/kddcup99/corrected.gz

--2022-03-20 08:15:24--  http://kdd.ics.uci.edu/databases/kddcup99/corrected.gz
Resolving kdd.ics.uci.edu (kdd.ics.uci.edu)... 128.195.1.86
Connecting to kdd.ics.uci.edu (kdd.ics.uci.edu)|128.195.1.86|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1409035 (1.3M) [application/x-gzip]
Saving to: ‘corrected.gz’

corrected.gz        100%[===================>]   1.34M  1.33MB/s    in 1.0s    

2022-03-20 08:15:25 (1.33 MB/s) - ‘corrected.gz’ saved [1409035/1409035]



In [52]:
! gunzip corrected.gz

In [53]:
! ls -l corrected*

-rw-r--r-- 1 root root 47251540 May 16  2000 corrected


In [54]:
! head -3 corrected

0,udp,private,SF,105,146,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0.00,0.00,0.00,0.00,1.00,0.00,0.00,255,254,1.00,0.01,0.00,0.00,0.00,0.00,0.00,0.00,normal.
0,udp,private,SF,105,146,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0.00,0.00,0.00,0.00,1.00,0.00,0.00,255,254,1.00,0.01,0.00,0.00,0.00,0.00,0.00,0.00,normal.
0,udp,private,SF,105,146,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0.00,0.00,0.00,0.00,1.00,0.00,0.00,255,254,1.00,0.01,0.00,0.00,0.00,0.00,0.00,0.00,normal.


In [55]:
test_df = sqlContext.read.format('com.databricks.spark.csv').\
options(header='false', inferschema='true').schema(schema).\
load('corrected')

In [56]:
test_df1 = test_df.repartition(60)

In [57]:
test_df2 = test_df1.cache()

In [58]:
test_df2.count()

311029

In [59]:
test_result = kMeansPredictionModel.\
transform(test_df2)

In [60]:
final_result = test_result.\
groupBy('attack','prediction').count()

##### น่าสนใจมากที่ ข้อมูลที่ไม่เคยนำมาใช้สร้างโมเดล แต่เมื่อนำมาถามโมเดลแล้ว ให้คำตอบที่ค่อนข้างแม่นยำ

In [61]:
final_result.filter(final_result['prediction'] == potentialAttack_group).\
orderBy(final_result['prediction'],ascending = False).show(150)

+--------+----------+------+
|  attack|prediction| count|
+--------+----------+------+
| normal.|         1|   588|
|  smurf.|         1|131114|
|ipsweep.|         1|     3|
+--------+----------+------+



##### มี smurf ที่หลุดไปจากกลุ่มนี้หรือไม่

In [62]:
final_result.filter(final_result['prediction'] != potentialAttack_group)\
.filter(sparkf.col('attack').startswith('smurf'))\
.orderBy(final_result['prediction'],ascending = False).show(150)

+------+----------+-----+
|attack|prediction|count|
+------+----------+-----+
|smurf.|        10|    8|
|smurf.|         3|32641|
|smurf.|         0|  328|
+------+----------+-----+

